In [1]:
import numpy as np
import youtube_dl
import subprocess

In [33]:
class MyLogger(object):
    def debug(self, msg):
        pass
    
    def warning(self, msg):
        pass
    
    def error(self, msg):
        print(msg)

class FrameExtractor():
    def __init__(self, output_dir):
        self.url_starter = 'youtube.com/watch?v='
        self.output_dir = output_dir
        self.v_info = None # v_id, start, end
        self.counter = 0
        
    def my_hook(self, d):
        if d['status'] == 'finished':
            self.counter += 1
            print('[{}]Done downloading, now converting ...'.format(self.counter))
            subprocess.call(['ffmpeg', \
                             '-ss', \
                             self.v_info[1], \
                             '-i', \
                             '{}{}_{}_{}.mp4'.format(self.output_dir, *self.v_info), \
                             '-vframes', \
                             '97', \
                             '-vf', \
                             'scale=352:288', \
                             '{}{}_{}_{}_%04d.png'.format(self.output_dir, *self.v_info) \
                            ])
            print('Done converting.')
            subprocess.call(['rm', \
                 '{}{}_{}_{}.mp4'.format(self.output_dir, *self.v_info) \
                ])
            

    def extract_frames(self, info):
        self.v_info = info

        ydl_opts = {
            'format': 'bestvideo[fps<=30]',
            'outtmpl': '{}%(id)s_{}_{}.%(ext)s'.format(self.output_dir, self.v_info[1], self.v_info[2]),
            'logger': MyLogger(),
            'progress_hooks': [self.my_hook],
        }

        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([self.url_starter + self.v_info[0]])

            

In [34]:
texts = np.genfromtxt('./train_ids/dl_demo.txt',dtype='str')
data = []
for t in texts:
    data.append(t.split('_')) # [id, start, end]

In [35]:
print(data)

[['08uqFlT6m00', '000227', '000237'], ['0CsAKXK4sgk', '000004', '000014'], ['0DiezTkGgR0', '000041', '000051']]


In [36]:
ext = FrameExtractor('./data/demo/')
for d in data:
    ext.extract_frames(d)

[1]Done downloading, now converting ...
Done converting.
[2]Done downloading, now converting ...
Done converting.
[3]Done downloading, now converting ...
Done converting.


In [37]:
texts = np.genfromtxt('./train_ids/vcii_test_ids.txt',dtype='str')
data = []
for t in texts:
    data.append(t.split('_')) # [id, start, end]

In [38]:
data[0]

['08uqFlT6m00', '000227', '000237']